### This notebook deals with all things Linear Regression. Starting with basic NN

In [1]:
import torch
from torch import nn
from torch.nn import functional as F 
from torch import optim

In [85]:
# Defining a simple NN model

class RegressionNN(nn.Module):
    def __init__(self, in_features, out_features, hidden_features):
        super(RegressionNN, self).__init__()
        self.lin1 = nn.Linear(in_features, hidden_features)  # first layer connected to hidden
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(hidden_features, out_features)  # hidden to out features

    def forward(self, x):
        x = self.lin1(x)
        # print(x)
        x = self.relu(x)
        # print(x)
        x = self.lin2(x)
        # print(x)
        return x
    
regmodel = RegressionNN(4, 1, 10)

In [36]:
from sklearn import datasets

x, y = datasets.make_regression(n_features=4, n_targets=1, 
                                n_samples=100, noise=25, random_state=123)
print(x.shape)
print(y.shape)

(100, 4)
(100,)


In [86]:
regmodel(rand_x[3])

tensor([0.4418], grad_fn=<ViewBackward0>)

In [51]:
rand_x = torch.randn(100, 4)
rand_y = 3 * rand_x[:][0] + 2 * rand_x[:][1] - 5.6 * rand_x[:][2]

In [62]:
rand_x.T.shape

torch.Size([4, 100])

In [67]:
rand_y = torch.tensor([x[0] * 3 + x[1] * 2.5 + x[2] * -0.5 + x[3] for x in rand_x], dtype=torch.float32)
rand_y.shape

torch.Size([100])

In [ ]:
rand_x.T[0] * 3 + rand_x.T[1] * 2.5 + rand_x.T[2] * -0.5 + rand_x.T[3]

In [ ]:
rand_y = rand_y.reshape(-1, 1)
rand_y

In [11]:
te = torch.arange(1, 5, 1, dtype=torch.float32)
te = te.reshape(-1, 1)
te[3]

tensor([4.])

In [37]:
from torch.utils.data import DataLoader, Dataset
import numpy as np

class Regset(Dataset):
    def __init__(self, x, y):
        self.x = torch.from_numpy(x.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.y = self.y.reshape(-1, 1)
        self.samples = self.x.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.samples

In [38]:
regset = Regset(x=x, y=y)
regset[1]

(tensor([-1.0075, -1.3058, -0.8828,  0.7898]), tensor([-281.0297]))

In [13]:
regloader = DataLoader(regset, 3)
iterable_loader = iter(regloader)
next(iterable_loader)

[tensor([[-1.4161, -0.6691,  1.6122, -0.2749],
         [-1.0075, -1.3058, -0.8828,  0.7898],
         [-1.4187, -0.8654, -1.3747,  0.9566]]),
 tensor([[ -65.6311],
         [-281.0297],
         [-337.7547]])]

In [92]:
criterion = nn.MSELoss()
optimiser = optim.Adam(params=regmodel.parameters(), lr=0.01)

In [ ]:
# 100 pass through the full data using dataloader
for ep in range(100):
    for ind, batch in enumerate(regloader):
        # print(batch[0].shape)  # torch.size([3, 4])
        pred_batch  = regmodel(batch[0])
        loss_cr = criterion(pred_batch, batch[1])
        # print(loss_cr.item())
        loss_cr.backward()  # backward pass happens with ease, even though there are multiple elements
        optimiser.step()
        optimiser.zero_grad()
    if ep % 5 == 0:
        print(f"Epoch is {ep} and loss is: {loss_cr.item():.2f}")

In [25]:
regset[0]

(tensor([-1.4161, -0.6691,  1.6122, -0.2749]), tensor([-65.6311]))

In [28]:
with torch.no_grad():
    test_pred = regmodel(regset[0][0])
    print(test_pred)
    print("compared to: ", regset[0][1])

tensor([0.0724])
compared to:  tensor([-65.6311])


In [ ]:
# 100 pass through full data using regular tensor
for ep in range(150):
    pred = regmodel(rand_x)
    # print('pred', pred)
    lo = criterion(pred, rand_y)
    # print(lo)
    lo.backward()
    optimiser.step()
    optimiser.zero_grad()
    if ep % 5 == 0:
        print(f"Epoch is {ep} and loss is {lo.item():.3f}")

In [94]:
with torch.no_grad():
    pred = regmodel(rand_x[0])
    print(pred, rand_y[0])

tensor([-5.2124]) tensor([-5.3929])
